Goal: replace random prediction with a neural network, however shitty. Also I'll clean things up a bit.

In [120]:
import torch

In [121]:
import torchvision.datasets as datasets
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor(),
])

train_data = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

In [122]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)


I guess I need to make an actual model here.

In [123]:
import torch.nn as nn
import torch.optim as optim

class Skynet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        return x

In [124]:
input_size = 28 ** 2
hidden_size = 16
output_size = 10

model = Skynet(input_size, hidden_size, output_size)


In [125]:
model.train(False)
preds = []
for features, labels in test_loader:
    with torch.no_grad():
        batch_preds = model(features)
        preds.extend(batch_preds.tolist())

In [126]:
preds_tensor = torch.tensor(preds)
category_preds = torch.argmax(preds_tensor, dim=1)

In [127]:
def check_accuracy(preds, actual):
    if len(actual) != len(preds):
        return -1
    return sum([int(actual[i] == preds[i]) for i in range(0, len(actual))]) / len(actual)

In [128]:
print(f"Accuracy: {check_accuracy(category_preds, test_data.targets)}")

Accuracy: 0.0854


So, I now have a neural network which can make predictions, though randomly, and I can check the accuracy of those predictions easily. Great!